In [1]:
using Pkg
Pkg.activate("."); 
Pkg.instantiate()

  Activating environment at `C:\Users\diersh\OneDrive - EWI an der Universität zu Köln\Dokumente\Promotion\ATIS3\data\Project.toml`


In [2]:
#ENV["CPLEX_STUDIO_BINARIES"] = "C:\\Program Files\\CPLEX_Studio1210\\cplex\\bin\\x86-64_win\\"
#Pkg.add("CPLEX")
#Pkg.build("CPLEX")

In [3]:
using JuMP, LinearAlgebra, Plots, StatsPlots, Statistics, DataFrames, XLSX, AxisArrays, Parameters, CSV, GLPK, BilevelJuMP, CPLEX#, CSV, GLPK, BilevelJuMP


(process:13232): GLib-GIO-WARNING **: 15:30:51.903: Unexpectedly, UWP app `c5e2524a-ea46-4f67-841f-6a9465d9d515_10.0.19041.1023_neutral_neutral_cw5n1h2txyewy' (AUMId `c5e2524a-ea46-4f67-841f-6a9465d9d515_cw5n1h2txyewy!App') supports 1 extensions but has no verbs

(process:13232): GLib-GIO-WARNING **: 15:30:52.003: Unexpectedly, UWP app `Microsoft.Windows.SecHealthUI_10.0.19041.1023_neutral__cw5n1h2txyewy' (AUMId `Microsoft.Windows.SecHealthUI_cw5n1h2txyewy!SecHealthUI') supports 1 extensions but has no verbs


In [4]:
using XLSX
input_path = "C:\\Users\\diersh\\OneDrive - EWI an der Universität zu Köln\\Dokumente\\Promotion\\ATIS3\\data\\"
xl = XLSX.readxlsx(input_path * "data_ATIS3.xlsx") 
data = Dict()

for s in XLSX.sheetnames(xl)
    data[s] = DataFrame(XLSX.gettable(xl[s]; infer_eltypes=true)...)
end
println("All files loaded")

All files loaded


In [7]:
data["time_series"][data["time_series"][:, :hour] .== 2, :]

LoadError: ArgumentError: unable to check bounds for indices of type Missing

In [14]:
names(data["time_series"])

26-element Vector{String}:
 "Begin"
 "p_da"
 "p_15"
 "p_id3"
 "fc_load_de"
 "ist_load_de"
 "fc_wind_de"
 "ist_wind_de"
 "fc_pv_de"
 "ist_pv_de"
 "ist_load_bw"
 "fc_load_bw"
 "ist_wind_bw"
 "fc_wind_bw"
 "ist_pv_bw"
 "fc_pv_bw"
 "error_wind_bw"
 "error_wind_de"
 "rel_error_wind_bw"
 "rel_error_wind_de"
 "int_ix"
 "year"
 "month"
 "day"
 "hour"
 "weekday"

In [ ]:
test=data["time_series"][:,"Begin"]

In [20]:
t=ones(1,50)
[t.*0.02]

1-element Vector{Matrix{Float64}}:
 [0.02 0.02 … 0.02 0.02]

### Irgendwelche Beispielwerte

In [ ]:
param = Dict(
    :a          => 0.85,  # lower bound for da bid
    :b          => 1.15,  # upper bound for da bid
    :FC         => data["time_series"][:,"fc_wind_bw"],  # Forecast

 :pda        => data["time_series"][:,"p_da"],   # Day ahead price
 :p15        => data["time_series"][:,"p_15"],   # 15 min price
 :pID3       => data["time_series"][:,"p_id3"],  # ID3 price

 :Ω => [1, 2], # index for up/down FCError Scenarios
 :Φ => collect(1:50), # index for second stage FCError scenarios

 :α          => [0.5, 0.5],        # Probability of Scenario Ω
 :β          => ones(1,50).*0.02,  # Probability of Scenario Φ
    
 :ω          => [-1, 1], #sign of FC Error
 :θ          => [0, 0.05, 0.1, 0.18, 0.25] # Error as a fraction of FC  
 )       

# to do FC* Scenarios to get q

In [ ]:
bm = BilevelModel(CPLEX.Optimizer, mode = BilevelJuMP.SOS1Mode())

In [ ]:
@variable(Upper(bm), param[:a]*param[:FC] <= qda <= param[:b]*param[:FC]) # day ahead bids regulatory bound
@variable(Lower(bm), q15[w in param[:Ω]])                 # q15 w/o strict limits
@variable(Lower(bm), qID3[w in param[:Ω],i in param[:Φ]])
@variable(Lower(bm), Qr[w in param[:Ω],i in param[:Φ]])
@variable(Lower(bm), FCE[w in param[:Ω],i in param[:Φ]])

@constraint(Lower(bm), ID3[w in param[:Ω],i in param[:Φ]], qID3[w,i] == Qr[w,i] -qda -q15[w])
@constraint(Lower(bm), QR[w in param[:Ω],i in param[:Φ]], Qr[w,i] == param[:FC] + FCE[w,i])
@constraint(Lower(bm), fce[w in param[:Ω],i in param[:Φ]], FCE[w,i] == param[:ω][w] * param[:θ][i]*param[:FC])


@objective(Upper(bm), Max, 
   sum(param[:α][w] * sum(param[:β][i] *               # sum over stochastic draws * probs
    (q15[w]*param[:p15] + qID3[w,i]*param[:pID3])       # wait and see decisions
    for i in param[:Φ]) for w in param[:Ω])              #iterating over all scenarios
    + qda*param[:pda])                           #here and now decision    


In [ ]:
#@objective(Lower(bm), Max, 
 #      q15[w]*p15 +
  #  sum(θ[i]*qID3[w,i]*pID3 for w in Ω, i in Φ))

## Vielleicht sollten wir doch nochmal überlegen, statt nem Bilevel model ein dynamisches mit non-anticipativity constraints zu benutzen
- als ich nochmal was nachschauen wollte, war das echt überall so..
- ansonsten müsst ihr nochmal schauen, wie man im Lower-Level mit Bilevel Jump (oder was auch immer) für die unterschiedlichen Realisationen des Vorzeichens einzeln optimiert - so jedenfalls nicht